UPDATED CODE

>> HIDING PASSWORD WHILE TYING

In [ ]:
!apt-get install libsqlite3-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsqlite3-dev is already the newest version (3.37.2-2ubuntu0.3).
libsqlite3-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
%%writefile Hospital_management_System_2.cpp
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <sqlite3.h>

void add_discharge_summary();
void view_discharge_summaries();
void add_prescription();
void add_user(const char *current_role);
void initialize_admin();

int initialize_database() {
    sqlite3 *db;
    char *err_msg = 0;

    int rc = sqlite3_open("hospital_management.db", &db);
    if (rc != SQLITE_OK) {
        printf("Cannot open database: %s\n", sqlite3_errmsg(db));
        sqlite3_close(db);
        return 1;
    }

    const char *sql =
        "CREATE TABLE IF NOT EXISTS users ("
        "id INTEGER PRIMARY KEY AUTOINCREMENT, "
        "username TEXT NOT NULL UNIQUE, "
        "password TEXT NOT NULL, "
        "role TEXT NOT NULL);"

        "CREATE TABLE IF NOT EXISTS prescriptions ("
        "id INTEGER PRIMARY KEY AUTOINCREMENT, "
        "patient_name TEXT, "
        "doctor_name TEXT, "
        "medications TEXT);"

        "CREATE TABLE IF NOT EXISTS discharge_summary ("
        "id INTEGER PRIMARY KEY AUTOINCREMENT, "
        "patient_name TEXT, "
        "summary TEXT);";

    rc = sqlite3_exec(db, sql, 0, 0, &err_msg);
    if (rc != SQLITE_OK) {
        printf("SQL error: %s\n", err_msg);
        sqlite3_free(err_msg);
        sqlite3_close(db);
        return 1;
    }

    sqlite3_close(db);
    initialize_admin();
    return 0;
}

void initialize_admin() {
    sqlite3 *db;
    char *err_msg = 0;

    int rc = sqlite3_open("hospital_management.db", &db);
    if (rc != SQLITE_OK) {
        printf("Cannot open database: %s\n", sqlite3_errmsg(db));
        sqlite3_close(db);
        return;
    }

    const char *sql =
        "INSERT OR IGNORE INTO users (id, username, password, role) "
        "VALUES (1, 'admin', 'admin123', 'admin');";

    rc = sqlite3_exec(db, sql, 0, 0, &err_msg);
    if (rc != SQLITE_OK) {
        printf("Error initializing admin: %s\n", err_msg);
        sqlite3_free(err_msg);
    }

    sqlite3_close(db);
}

int authenticate_user(const char *username, const char *password, char *role) {
    sqlite3 *db;
    sqlite3_stmt *stmt;
    int rc = sqlite3_open("hospital_management.db", &db);

    if (rc != SQLITE_OK) {
        printf("Cannot open database: %s\n", sqlite3_errmsg(db));
        return 0;
    }

    const char *sql = "SELECT role FROM users WHERE username=? AND password=?";
    rc = sqlite3_prepare_v2(db, sql, -1, &stmt, 0);

    sqlite3_bind_text(stmt, 1, username, -1, SQLITE_STATIC);
    sqlite3_bind_text(stmt, 2, password, -1, SQLITE_STATIC);

    int authenticated = 0;
    if (sqlite3_step(stmt) == SQLITE_ROW) {
        strcpy(role, (const char *)sqlite3_column_text(stmt, 0));
        authenticated = 1;
    }

    sqlite3_finalize(stmt);
    sqlite3_close(db);

    return authenticated;
}

void add_user(const char *current_role) {
    if (strcmp(current_role, "admin") != 0) {
        printf("Access denied. Only admins can add users.\n");
        return;
    }

    sqlite3 *db;
    char *err_msg = 0;

    char username[50], password[50], role[20];
    printf("Enter username: ");
    scanf("%s", username);
    printf("Enter password: ");
    scanf("%s", password);
    printf("Enter role (admin/staff): ");
    scanf("%s", role);

    int rc = sqlite3_open("hospital_management.db", &db);
    if (rc != SQLITE_OK) {
        printf("Cannot open database: %s\n", sqlite3_errmsg(db));
        return;
    }

    char sql[256];
    snprintf(sql, sizeof(sql),
        "INSERT INTO users (username, password, role) VALUES ('%s', '%s', '%s');",
        username, password, role);

    rc = sqlite3_exec(db, sql, 0, 0, &err_msg);
    if (rc != SQLITE_OK) {
        printf("Error adding user: %s\n", err_msg);
        sqlite3_free(err_msg);
    } else {
        printf("User added successfully.\n");
    }

    sqlite3_close(db);
}

void add_prescription() {
    char patient_name[100], doctor_name[100], medications[255];

    printf("Enter patient name: ");
    scanf("%s", patient_name);
    printf("Enter doctor name: ");
    scanf("%s", doctor_name);
    printf("Enter medications: ");
    scanf("%s", medications);

    sqlite3 *db;
    char *err_msg = 0;
    sqlite3_open("hospital_management.db", &db);

    char sql[1024];
    snprintf(sql, sizeof(sql),
             "INSERT INTO prescriptions (patient_name, doctor_name, medications) "
             "VALUES ('%s', '%s', '%s');",
             patient_name, doctor_name, medications);

    if (sqlite3_exec(db, sql, 0, 0, &err_msg) != SQLITE_OK) {
        printf("Error adding prescription: %s\n", err_msg);
        sqlite3_free(err_msg);
    } else {
        printf("Prescription added successfully!\n");
    }

    sqlite3_close(db);
}

void add_discharge_summary() {
    char patient_name[100];
    char summary[255];

    printf("Enter patient name: ");
    scanf(" %[^\n]", patient_name);
    printf("Enter discharge summary: ");
    scanf(" %[^\n]", summary);

    sqlite3 *db;
    char *err_msg = 0;
    sqlite3_open("hospital_management.db", &db);

    char sql[512];
    snprintf(sql, sizeof(sql),
             "INSERT INTO discharge_summary (patient_name, summary) "
             "VALUES ('%s', '%s');",
             patient_name, summary);

    if (sqlite3_exec(db, sql, 0, 0, &err_msg) != SQLITE_OK) {
        printf("Error adding discharge summary: %s\n", err_msg);
        sqlite3_free(err_msg);
    } else {
        printf("Discharge summary added successfully for %s.\n", patient_name);
    }

    sqlite3_close(db);
}

void view_discharge_summaries() {
    sqlite3 *db;
    sqlite3_stmt *stmt;

    int rc = sqlite3_open("hospital_management.db", &db);
    if (rc != SQLITE_OK) {
        printf("Cannot open database: %s\n", sqlite3_errmsg(db));
        return;
    }

    const char *sql = "SELECT id, patient_name, summary FROM discharge_summary";

    rc = sqlite3_prepare_v2(db, sql, -1, &stmt, 0);
    if (rc != SQLITE_OK) {
        printf("Failed to fetch discharge summaries: %s\n", sqlite3_errmsg(db));
        sqlite3_close(db);
        return;
    }

    printf("\nDischarge Summaries:\n");
    printf("------------------------------------------\n");
    while (sqlite3_step(stmt) == SQLITE_ROW) {
        int id = sqlite3_column_int(stmt, 0);
        const char *patient_name = (const char *)sqlite3_column_text(stmt, 1);
        const char *summary = (const char *)sqlite3_column_text(stmt, 2);

        printf("ID: %d\n", id);
        printf("Patient Name: %s\n", patient_name);
        printf("Summary: %s\n", summary);
        printf("------------------------------------------\n");
    }

    sqlite3_finalize(stmt);
    sqlite3_close(db);
}

int main() {
    if (initialize_database() != 0) {
        printf("Database initialization failed.\n");
        return 1;
    }

    char username[50], password[50], role[20];

    printf("Hospital Management System\n");
    printf("Login\n");
    printf("Username: ");
    scanf("%s", username);
    printf("Password: ");
    scanf("%s", password);

    if (!authenticate_user(username, password, role)) {
        printf("Invalid login credentials.\n");
        return 1;
    }

    printf("Welcome, %s (%s)!\n", username, role);

    int choice;
    while (1) {
        printf("\nMenu:\n");
        if (strcmp(role, "admin") == 0) {
            printf("1. Add User\n");
        }
        printf("2. Add Prescription\n");
        printf("3. Add Discharge Summary\n");
        printf("4. View Discharge Summaries\n");
        printf("5. Exit\n");
        printf("Enter choice: ");
        scanf("%d", &choice);

        switch (choice) {
            case 1:
                if (strcmp(role, "admin") == 0) {
                    add_user(role);
                } else {
                    printf("Invalid choice.\n");
                }
                break;
            case 2:
                add_prescription();
                break;
            case 3:
                add_discharge_summary();
                break;
            case 4:
                view_discharge_summaries();
                break;
            case 5:
                printf("Exiting...\n");
                return 0;
            default:
                printf("Invalid choice.\n");
        }
    }

    return 0;
}


Overwriting Hospital_management_System_2.cpp


In [ ]:
!g++ Hospital_management_System_2.cpp -o HospitalManagementSystem -lsqlite3
!./HospitalManagementSystem

Hospital Management System
Login
Username: ^C
